# Sample of calculating stress (with timing)

## Describing variables

## Code

In [ ]:
import fenics as fe
import matplotlib.pyplot as plt
import numpy as np
import ufl

fe.set_log_level(13)

# --------------------
# Функции и классы
# --------------------
def bottom(x, on_boundary):
    return (on_boundary and fe.near(x[1], 0.0))


# Функция деформации (Strain function)
def epsilon(u):
    return 0.5*(fe.grad(u) + fe.grad(u).T)


# Функция напряжения (Stress function)
def sigma(u):
    return lmbda*fe.div(u)*fe.Identity(2) + 2*mu*epsilon(u)


def calc_linear():
    fe.solve(a == l, u_small, bc)
    return u_small((2.5, 5.0))[1]


def calc_nonlinear():
    Form = fe.derivative(psi, u_large, u_test)
    Jac = fe.derivative(Form, u_large, u_tr)

    problem = fe.NonlinearVariationalProblem(Form, u_large, bc, Jac)
    solver = fe.NonlinearVariationalSolver(problem)
    prm = solver.parameters
    prm["newton_solver"]["error_on_nonconvergence"] = False
    solver.solve()

    return u_large((2.5, 5.0))[1]


# --------------------
# Параметры
# --------------------

E = float(input("Введите модуль Юнга: ") or "5.0e6")  # Модуль Юнга
mu = 0.5*E  # Модуль сдвига
rho_0 = float(input("Введите плотность материала: ") or "800")  # Плотность материала

lmbda = float(input("Введите первый параметр Ламе (лямбда): ") or "1.0e6")  # Первый параметр Ламе

# l_x, l_y = 5.0, 5.0  # Размеры области
# n_x, n_y = 500, 500  # Количество элементов

b_int = float(input("Введите величину приложенной силы: "))  # Величина приложенной силы

# --------------------
# Геометрия
# --------------------

mesh = fe.Mesh("rectangle_mesh.xml")  # Загрузка сетки из файла

# Определение границы для условия Неймана

boundaries = fe.MeshFunction("size_t", mesh, mesh.topology().dim() - 1)
boundaries.set_all(0)

top = fe.AutoSubDomain(lambda x: fe.near(x[1], 5.0))
top.mark(boundaries, 1)

ds = fe.ds(subdomain_data=boundaries)  # Измерение для интеграции по границе

# --------------------
# Пространства функций
# --------------------

V = fe.VectorFunctionSpace(mesh, "CG", 1)  # Пространство векторных функций

u_tr = fe.TrialFunction(V)  # Функция-тест
u_test = fe.TestFunction(V)  # Функция-испытание
u_small = fe.Function(V)  # Линейное решение
u_large = fe.Function(V)  # Нелинейное решение

g = fe.Expression((0.0, "t"), t=0, degree=0)  # Приложенная сила
b = fe.Constant((0.0, b_int))  # Вектор приложенной силы
N = fe.Constant((0.0, 1.0))  # Единичный вектор

aa, bb, cc, dd, ee = 0.5*mu, 0.0, 0.0, mu, -1.5*mu  # Коэффициенты модели материала


start_time = float(input("Введите значение начальной времени: ") or "0.0") 
end_time = float(input("Введите значение конечной времени: ")) 
pseudo_time = np.linspace(start_time, end_time, 100)  # Массив временных значений

# --------------------
# Граничные условия
# --------------------

bc = fe.DirichletBC(V, fe.Constant((0.0, 0.0)), bottom)  # Условие Дирихле на нижней границе

# --------------------
# Слабая формулировка
# --------------------

I = fe.Identity(2)
F = I + fe.grad(u_large)  # Тензор деформации
C = F.T*F  # Тензор Коши-Грина
J = fe.det(F)  # Определитель тензора деформации

n = fe.dot(ufl.cofac(F), N)
surface_def = fe.sqrt(fe.inner(n, n))

psi = (aa*fe.inner(F, F) + ee - dd*fe.ln(J))*fe.dx - rho_0*J*fe.dot(b, u_large)*fe.dx + surface_def*fe.inner(g, u_large)*ds(1)

a = fe.inner(sigma(u_tr), epsilon(u_test))*fe.dx
l = rho_0*fe.dot(b, u_test)*fe.dx - fe.inner(g, u_test)*ds(1)

# --------------------
# Цикл по времени
# --------------------

u_lin = []
u_nlin = []
for ti in pseudo_time:
    g.t = ti
    u_lin.append(calc_linear()*1000.0)
    u_nlin.append(calc_nonlinear()*1000.0)




# --------------------
# Пост-обработка
# --------------------

fe.plot(u_small, mode="displacement")
plt.show()

fe.plot(u_large, mode="displacement")
plt.show()

plt.plot(pseudo_time, u_lin, label="линейный")
plt.plot(pseudo_time, u_nlin, label="нелинейный")
plt.xlabel("псевдо-время")
plt.ylabel("смещение [мм]")
plt.legend()
plt.show()